In [1]:
!pip install tensorflow


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow
!pip install transformers
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install datasets


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import create_optimizer, AdamWeightDecay

d:\Projects\docker-signalone\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Initialize the BART tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')
model = TFAutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model='facebook/bart-base', return_tensors="tf")

All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [8]:
import pandas as pd
from datasets import Dataset

# Read the CSV file
df = pd.read_csv('D:\Projects\docker-signalone\logDataset\dataset.csv')

# Define the prefix for summarization
prefix = "summarize: "

# Define the preprocess function
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["logs"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Convert the data to a Dataset object
dataset = Dataset.from_pandas(df)

# Tokenize the input logs and summaries
tokenized_data = dataset.map(preprocess_function, batched=True)

# Print the first example
print(tokenized_data[0])

Map: 100%|██████████| 1056/1056 [00:00<00:00, 1941.10 examples/s]

{'logs': 'Docker container failed to start. Error code: 137.\nFunction name: startContainer.\nError message: Out of memory.', 'summary': 'The Docker container failed to start due to an out of memory error (Error code: 137) in the startContainer function.', 'input_ids': [0, 18581, 3916, 2072, 35, 42221, 11255, 1447, 7, 386, 4, 37943, 3260, 35, 22619, 4, 50118, 47802, 766, 35, 386, 48557, 4, 50118, 30192, 1579, 35, 2548, 9, 3783, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 133, 42221, 11255, 1447, 7, 386, 528, 7, 41, 66, 9, 3783, 5849, 36, 30192, 3260, 35, 22619, 43, 11, 5, 386, 48557, 5043, 4, 2]}


In [9]:
print(tokenized_data[1000])

{'logs': "An exception occurred while starting the Docker service. Error code 654. The function 'startDockerService' threw an error. Check system logs for more details.", 'summary': "Failed to start the Docker service due to exception with error code 654 in the 'startDockerService' function. Please review system logs for further information.", 'input_ids': [0, 18581, 3916, 2072, 35, 660, 8219, 2756, 150, 1158, 5, 42221, 544, 4, 37943, 3260, 231, 4283, 4, 20, 5043, 128, 13124, 495, 23145, 32537, 108, 4021, 41, 5849, 4, 4254, 467, 24113, 13, 55, 1254, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 597, 13355, 7, 386, 5, 42221, 544, 528, 7, 8219, 19, 5849, 3260, 231, 4283, 11, 5, 128, 13124, 495, 23145, 32537, 108, 5043, 4, 3401, 1551, 467, 24113, 13, 617, 335, 4, 2]}


In [10]:
from sklearn.model_selection import train_test_split

# Split the tokenized_data into training and validation datasets
train_data, val_data = train_test_split(tokenized_data, test_size=0.2, random_state=42)

# Convert the training and validation datasets to Dataset objects
tokenized_training_dataset = Dataset.from_dict(train_data)
tokenized_validation_dataset = Dataset.from_dict(val_data)

In [11]:
import numpy as np

In [12]:
np.object = object

tf_train_set = model.prepare_tf_dataset(
    tokenized_training_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_validation_dataset,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [13]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [14]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=5)

Epoch 1/5

52/52 [==============================] - 1187s 22s/step - loss: 2.1295 - val_loss: 1.4002
Epoch 2/5
52/52 [==============================] - 1029s 20s/step - loss: 1.5589 - val_loss: 1.2100
Epoch 3/5
52/52 [==============================] - 1217s 24s/step - loss: 1.3727 - val_loss: 1.1205
Epoch 4/5
52/52 [==============================] - 1025s 18s/step - loss: 1.2238 - val_loss: 1.0446
Epoch 5/5
52/52 [==============================] - 1125s 22s/step - loss: 1.1150 - val_loss: 1.0105


In [20]:
model.save_pretrained('logBART')
